<a href="https://colab.research.google.com/github/DoItSon/Dacon/blob/main/%EB%8D%B0%EC%9D%B4%EC%BD%98%20%EC%B5%9C%EC%A2%85%EC%88%9C%EC%9C%84%2029%EC%9C%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import

In [ ]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer ,TfidfVectorizer

import torch

from tqdm.auto import tqdm
from sklearn.metrics import f1_score

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

# mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_PATH = "/content/drive/MyDrive/"
SEED = 42

In [ ]:
if not os.path.isdir("image"):
    !cp "{DATA_PATH}open.zip" "open.zip"
    !unzip -qq "open.zip"

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train

,id,img_path,overview,cat1,cat2,cat3
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식
...,...,...,...,...,...,...
16981,TRAIN_16981,./image/train/TRAIN_16981.jpg,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,음식,음식점,한식
16982,TRAIN_16982,./image/train/TRAIN_16982.jpg,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,숙박,숙박시설,모텔
16983,TRAIN_16983,./image/train/TRAIN_16983.jpg,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,숙박,숙박시설,모텔
16984,TRAIN_16984,./image/train/TRAIN_16984.jpg,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,레포츠,육상 레포츠,"야영장,오토캠핑장"


In [ ]:
test

,id,img_path,overview
0,TEST_00000,./image/test/TEST_00000.jpg,신선한 재료로 정성을 다해 만들었다. 늘 변함없는 맛과 서비스로 모실것을 약속한다.
1,TEST_00001,./image/test/TEST_00001.jpg,"청청한 해역 등량만과 율포해수욕장이 한눈에 내려다 보이는 위치에 있으며, 막 잡은 ..."
2,TEST_00002,./image/test/TEST_00002.jpg,장터설렁탕은 남녀노소 누구나 즐길 수 있는 전통 건강식으로 좋은 재료와 전통 조리방...
3,TEST_00003,./image/test/TEST_00003.jpg,다양한 형태의 청소년수련활동을 제공함으로써 청소년들이 민주사회의 주역이 될 수 있도...
4,TEST_00004,./image/test/TEST_00004.jpg,팔공산은 경산시의 북쪽에 위치한 해발 1192.3 m의 높은 산으로 신라시대에는 중...
...,...,...,...
7275,TEST_07275,./image/test/TEST_07275.jpg,"막국수와 수육을 주메뉴로 하며, 넓은 주차장이 마련되어 있어 주차하기 편리하다.<br>"
7276,TEST_07276,./image/test/TEST_07276.jpg,"통진두레문화센터는 우리고유의 전통무형문화와 민속예술을 계승/발전 시키고, 다양한 문..."
7277,TEST_07277,./image/test/TEST_07277.jpg,"수도권에서 가까운 위치, 문산천을 따라 걷는 산책코스, 한여름 더위를 날려버릴 시원..."
7278,TEST_07278,./image/test/TEST_07278.jpg,\n전남 구례군 관산리에 위치한 노고단 게스트하우스&호텔은 지리산을 파노라마로 관망...


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16986 entries, 0 to 16985
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        16986 non-null  object
 1   img_path  16986 non-null  object
 2   overview  16986 non-null  object
 3   cat1      16986 non-null  object
 4   cat2      16986 non-null  object
 5   cat3      16986 non-null  object
dtypes: object(6)
memory usage: 796.3+ KB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7280 entries, 0 to 7279
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7280 non-null   object
 1   img_path  7280 non-null   object
 2   overview  7280 non-null   object
dtypes: object(3)
memory usage: 170.8+ KB


In [ ]:
train.nunique()

id          16986
img_path    16986
overview    16973
cat1            6
cat2           18
cat3          128
dtype: int64

In [ ]:
test.nunique()

id          7280
img_path    7280
overview    7280
dtype: int64

In [ ]:
le = LabelEncoder()
train["target"] = le.fit_transform(train['cat3'].to_numpy())
train

,id,img_path,overview,cat1,cat2,cat3,target
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구,120
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프,8
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식,118
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식,118
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식,118
...,...,...,...,...,...,...,...
16981,TRAIN_16981,./image/train/TRAIN_16981.jpg,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,음식,음식점,한식,118
16982,TRAIN_16982,./image/train/TRAIN_16982.jpg,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,숙박,숙박시설,모텔,31
16983,TRAIN_16983,./image/train/TRAIN_16983.jpg,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,숙박,숙박시설,모텔,31
16984,TRAIN_16984,./image/train/TRAIN_16984.jpg,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,레포츠,육상 레포츠,"야영장,오토캠핑장",73


In [ ]:
!pip install kiwipiepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.1 MB/s 
     |████████████████████████████████| 30.5 MB 1.4 MB/s 
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.14.0-py3-none-any.whl size=30560039 sha256=f00d1146b97893c1a0d0c922fac5988f9cc6048571748a861687e077954c20dd
  Stored in directory: /root/.cache/pip/wheels/97/36/50/1fc99e4a3ce636ed30739cc2cb333b36e6e3778f365f1adf77
Successfully built kiwipiepy-model


In [ ]:
from kiwipiepy import Kiwi

kiwi = Kiwi(model_type='sbg')
kiwi.prepare()

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.0 MB/s 
     |████████████████████████████████| 163 kB 71.9 MB/s 
     |████████████████████████████████| 7.6 MB 60.3 MB/s 


In [ ]:
model_name = "kykim/bert-kor-base"

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/725 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/344k [00:00<?, ?B/s]

In [ ]:
train_overview = np.array(train["overview"])
test_overview = np.array(test["overview"])
target = train.target.to_numpy()

In [ ]:
train_overview[0]

'소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 올리고 있으며 바다낚시터로도 유명하다. 항 주변에 설치된 양식장들은 섬사람들의 부지런한 생활상을 고스 란히 담고 있으며 일몰 때 섬의 정경은 바다의 아름다움을 그대로 품고 있는 듯하다. 또한, 섬에는 각시여 전설, 도둑바위 등의 설화가 전해 내려오고 있으며, 매년 정월 풍어제 풍속이 이어지고 있다.<br>'

In [ ]:
token = tokenizer(train_overview[0], add_special_tokens=True,padding="max_length", truncation=True)
token

{'input_ids': [2, 5035, 8102, 8200, 8078, 26959, 4971, 13969, 18416, 8010, 17684, 23379, 8010, 24809, 17153, 8018, 2482, 28747, 8051, 14261, 14617, 19576, 8051, 18462, 14754, 15721, 20158, 22236, 8144, 28602, 2016, 7670, 18874, 24106, 28747, 8020, 14022, 4971, 16065, 14086, 23448, 8112, 14686, 14331, 25115, 3685, 8100, 19345, 14754, 5929, 8576, 3463, 4971, 8107, 6035, 23097, 37597, 31056, 14624, 30006, 13979, 22713, 2016, 14254, 2014, 4971, 13994, 2103, 8118, 8088, 22122, 2014, 29151, 29716, 14850, 4964, 15247, 23685, 34607, 8014, 14754, 2014, 17922, 6035, 8063, 7557, 8082, 8031, 7557, 25586, 33758, 13984, 2016, 2030, 20958, 2032, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self ,tokenizer , x, y = None ): 
        self.tokenizer = tokenizer
        self.x = x
        self.y = y
    def __len__(self): 
        return self.x.shape[0]
    def __getitem__(self, idx): 
        item = {}
        item["x"] = self.__tokenizer(self.x[idx]) 
        if self.y is not None:
            item["y"] = torch.tensor(self.y[idx])
        return item
    def __tokenizer(self,text):
        inputs = self.tokenizer(text, add_special_tokens=True,padding="max_length", truncation=True)
        for k, v in inputs.items(): 
            inputs[k] = torch.LongTensor(v) 
        return inputs

In [ ]:
dt = MyDataset(tokenizer,train_overview,target)
dl = torch.utils.data.DataLoader(dt, batch_size=2,shuffle=False) 
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[    2,  5035,  8102,  ...,     0,     0,     0],
         [    2, 16403, 22624,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}, 'y': tensor([120,   8])}

In [ ]:
model = AutoModel.from_pretrained(model_name)

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
batch["x"].keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
outputs = model(**batch["x"])
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [ ]:
model.config.hidden_size

768

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, model_name): 
        super().__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.output_layer = torch.nn.Linear(self.model.config.hidden_size, 128)
    def forward(self, x):
        x = self.model(**x) 
        x = self.output_layer(x[1])
        return x

In [ ]:
train.target.nunique()

128

In [ ]:
def train_loop(dataloader,model,loss_fn,optimizer,device):
    epoch_loss = 0 
    model.train() 
    for batch in tqdm(dataloader): 
        pred = model(batch["x"].to(device)) 
        loss = loss_fn(pred, batch["y"].to(device)) 
        
        optimizer.zero_grad() 
        loss.backward()  
        optimizer.step() 
        
        epoch_loss += loss.item() 

    epoch_loss /= len(dataloader) 

    return epoch_loss 

In [ ]:
@torch.no_grad() 
def test_loop(dataloader,model,loss_fn,device): 
    epoch_loss = 0
    model.eval() 

    
    pred_list = []
    softmax = torch.nn.Softmax(dim=1) 

    for batch in tqdm(dataloader):
        
        pred = model(batch["x"].to(device))
        if batch.get("y") is not None: 
            loss = loss_fn(pred, batch["y"].to(device))
            epoch_loss += loss.item()
        
        pred = softmax(pred)
        pred = pred.to("cpu").numpy() 
        pred_list.append(pred)

    epoch_loss /= len(dataloader)

    pred = np.concatenate(pred_list) 
    return epoch_loss , pred 

In [ ]:
n_splits = 5
batch_size = 4
epochs = 100
loss_fn = torch.nn.CrossEntropyLoss() # 다중분류 손실객체

In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
# from sklearn.model_selection import StratifiedKFold

cv = KFold(n_splits=n_splits,shuffle=True, random_state=SEED)

In [ ]:
is_holdout = False
seed_everything(SEED)
best_score_list = []
for i,(tri,vai) in enumerate(cv.split(train_overview)):

    model = Net(model_name).to(device)
    optimizer = torch.optim.NAdam(model.parameters(),lr=0.00003)

    train_dt = MyDataset(tokenizer,train_overview[tri],target[tri])
    valid_dt = MyDataset(tokenizer,train_overview[vai],target[vai])
    train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True)
    valid_dl = torch.utils.data.DataLoader(valid_dt, batch_size=batch_size,shuffle=False)

    best_score = 0
    patience = 0

    for epoch in range(epochs):
        
        train_loss = train_loop(train_dl, model, loss_fn,optimizer,device )
        valid_loss , pred = test_loop(valid_dl, model, loss_fn,device  )
        
        pred = np.argmax(pred, axis=1) 
        true = target[vai] 
        score = f1_score(true, pred , average="weighted")
        print(train_loss,valid_loss,score)
        patience += 1
        if best_score < score:
            patience = 0
            best_score = score
            torch.save(model.state_dict(),f"{DATA_PATH}model_{i}.pth")

        if patience == 2:
            break
    print(f"Fold ({i}), BEST F1: {best_score}")
    best_score_list.append(best_score)

    torch.cuda.empty_cache()

    if is_holdout: 
        break

# Fold (2), BEST F1: 0.8174915575894353
# Fold (3), BEST F1: 0.8249715383866895

In [ ]:
np.mean(best_score_list)

In [ ]:
test_dt = MyDataset(tokenizer,test_overview)
test_dl = torch.utils.data.DataLoader(test_dt, batch_size=batch_size, shuffle=False)

pred_list = []
for i in range(n_splits):
    
    model = Net(model_name).to(device)
    state_dict = torch.load(f'{DATA_PATH}/모델 폴더/123456789/model_{i}.pth') 
    model.load_state_dict(state_dict)
        
    _ , pred = test_loop(test_dl, model, loss_fn,device  )
    pred_list.append(pred)
    
    # if is_holdout:
    #     break

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/1820 [00:00<?, ?it/s]

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/1820 [00:00<?, ?it/s]

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/1820 [00:00<?, ?it/s]

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/1820 [00:00<?, ?it/s]

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/1820 [00:00<?, ?it/s]

In [ ]:
np.save(f'{DATA_PATH}/모델 폴더/123456789/pred_list(son).npy',pred_list)

In [ ]:
# callfile = np.load(f'{DATA_PATH}')

In [ ]:
pred = np.mean(pred_list,axis=0) 
pred = np.argmax(pred, axis=1) 
pred

array([118, 118, 118, ...,  73,  31,  44])

In [ ]:
submit = pd.read_csv("sample_submission.csv")
submit["cat3"] = le.inverse_transform(pred)
submit

,id,cat3
0,TEST_00000,한식
1,TEST_00001,한식
2,TEST_00002,한식
3,TEST_00003,수련시설
4,TEST_00004,산
...,...,...
7275,TEST_07275,한식
7276,TEST_07276,공연장
7277,TEST_07277,"야영장,오토캠핑장"
7278,TEST_07278,모텔


In [ ]:
submit.to_csv(f'{DATA_PATH}submit.csv', index=False)